In [1]:
import os
import re
import shutil
import threading
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from PyPDF2 import PdfReader
from tqdm import tqdm
import pandas as pd
import win32com.client as win32
import win32com


In [5]:

DIRETORIO = r"C:\dev\Itau\Superveniência\superveniencia_piscofins\Output"
DIRETORIO_INVALIDO = r"C:\dev\Itau\Superveniência\superveniencia_piscofins\Output\Final\Inválido"
THREAD_LIMIT = 30


## Definitions

### verificar_e_mover_arquivo

In [6]:
def verificar_e_mover_arquivo(arquivo, diretorio_invalido):
    nome_arquivo = os.path.splitext(os.path.basename(arquivo))[0][1:].lstrip('0')
    df = pd.read_excel(arquivo, header=None, dtype=str)
    if str(df.iloc[1, 2]) != nome_arquivo and str(df.iloc[0, 2]) != nome_arquivo:
        os.makedirs(diretorio_invalido, exist_ok=True)
        shutil.move(arquivo, os.path.join(diretorio_invalido, os.path.basename(arquivo)))


### renomear_arquivos

In [7]:
def renomear_arquivos(diretorio):
    for arquivo in os.listdir(diretorio):
        if '.' in arquivo:
            nome, ext = arquivo.split('.', 1)
            letra = nome[0]
            numeros = nome[1:].zfill(7)
            novo_nome = f"{letra}{numeros}.{ext}"
            try:
                os.rename(os.path.join(diretorio, arquivo), os.path.join(diretorio, novo_nome))
            except Exception as e:
                pass


### linha_vazia

In [8]:
def linha_vazia(ws, row_index):
    if ws.row_dimensions[row_index].hidden:
        for c in range(1, ws.max_column + 1):
            ws.cell(row=row_index, column=c).value = None
        return True
    return all(ws.cell(row=row_index, column=c).value in [None, 0, 0.0] for c in range(1, ws.max_column + 1))


### coluna_vazia

In [9]:
def coluna_vazia(ws, col_index):
    if ws.column_dimensions[get_column_letter(col_index)].hidden:
        return True
    return all(ws.cell(row=r, column=col_index).value in [None, 0, 0.0] for r in range(1, ws.max_row + 1))


### processar_arquivo

In [10]:
def processar_arquivo(caminho_arquivo):
    try:
        wb = load_workbook(caminho_arquivo)
        for sheet in wb.worksheets:
            if not any(sheet.row_dimensions[row].hidden for row in range(1, sheet.max_row + 1)):
                continue

            for r in range(sheet.max_row, 0, -1):
                if linha_vazia(sheet, r):
                    if r <= sheet.max_row - 2:
                        sheet.delete_rows(r)

            for c in range(sheet.max_column, 0, -1):
                if coluna_vazia(sheet, c):
                    if c <= sheet.max_column - 2:
                        sheet.delete_cols(c)

            for row in range(1, sheet.max_row + 1):
                sheet.row_dimensions[row].hidden = False
            for col in range(1, sheet.max_column + 1):
                sheet.column_dimensions[get_column_letter(col)].hidden = False

        wb.save(caminho_arquivo)
    except Exception as e:
        print(f"Erro ao processar o arquivo {caminho_arquivo}: {e}")


### check_pdf

In [11]:
def check_pdf(input_file):
    output_file = os.path.splitext(input_file)[0] + ".pdf"
    if os.path.isfile(output_file):
        try:
            with open(output_file, 'rb') as f:
                pdf = PdfReader(f)
                return len(pdf.pages) == 1
        except:
            return False
    return False


### xlsx_to_pdf_one_page

In [12]:
def xlsx_to_pdf_one_page(input_file):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.Visible = False
    output_file = os.path.splitext(input_file)[0] + ".pdf"

    try:
        wb = excel.Workbooks.Open(input_file)
    except Exception as e:
        print(f"Erro ao abrir {input_file}: {e}")
        return

    ws = wb.Worksheets(1)
    last_column = ws.UsedRange.Columns.Count
    last_row = 1

    for row in range(1, ws.UsedRange.Rows.Count + 1):
        borders = ws.Cells(row, 4).Borders
        for i in range(5, 13):
            if borders(i).Color != 0.0 or borders(i).LineStyle != -4142:
                last_row = row
                break

    for col in range(1, ws.UsedRange.Columns.Count + 1):
        borders = ws.Cells(5, col).Borders
        for i in range(5, 13):
            if borders(i).Color != 0.0 or borders(i).LineStyle != -4142:
                last_column = col
                break

    def col_letra(i):
        letra = ''
        while i > 0:
            i, r = divmod(i - 1, 26)
            letra = chr(65 + r) + letra
        return letra

    ws.PageSetup.PrintArea = ws.Range(f"B1:{col_letra(last_column)}{last_row}").Address
    ws.PageSetup.Zoom = False
    ws.PageSetup.FitToPagesWide = 1
    ws.PageSetup.FitToPagesTall = 1

    wb.ExportAsFixedFormat(0, output_file)
    wb.Close(False)
    excel.Quit()


## Processamento

In [13]:
# Limpar cache do COM (apenas 1x por ambiente)
shutil.rmtree(os.path.join(win32com.__gen_path__), ignore_errors=True)

# Criar pasta de inválidos
os.makedirs(DIRETORIO_INVALIDO, exist_ok=True)

# 1. Verificar arquivos inválidos
arquivos = os.listdir(DIRETORIO)
threads = []
print("1. Verificando arquivos inválidos...")
for idx, arquivo in enumerate(tqdm(arquivos, desc="Verificando")):
    if arquivo.endswith(".pdf"):
        continue
    path = os.path.join(DIRETORIO, arquivo)
    t = threading.Thread(target=verificar_e_mover_arquivo, args=(path, DIRETORIO_INVALIDO))
    t.start()
    threads.append(t)
    if len(threads) >= THREAD_LIMIT or idx == len(arquivos) - 1:
        for t in threads:
            t.join()
        threads = []

# 2. Renomear
print("2. Renomeando arquivos...")
renomear_arquivos(DIRETORIO)

# 3. Limpar planilhas
print("3. Limpando planilhas...")
arquivos = [f for f in os.listdir(DIRETORIO) if f.lower().endswith('.xlsx')]
threads = []
for idx, arquivo in enumerate(tqdm(arquivos, desc="Limpando")):
    path = os.path.join(DIRETORIO, arquivo)
    t = threading.Thread(target=processar_arquivo, args=(path,))
    t.start()
    threads.append(t)
    if len(threads) >= THREAD_LIMIT or idx == len(arquivos) - 1:
        for t in threads:
            t.join()
        threads = []

# 4. PDF
print("4. Convertendo para PDF...")
arquivos = sorted(
    [f for f in os.listdir(DIRETORIO) if f.endswith(".xlsx") and not check_pdf(os.path.join(DIRETORIO, f))]
)
for arquivo in tqdm(arquivos, desc="Convertendo"):
    try:
        xlsx_to_pdf_one_page(os.path.abspath(os.path.join(DIRETORIO, arquivo)))
    except Exception as e:
        print(f"Erro ao converter {arquivo}: {e}")

print("✅ Processo finalizado.")


1. Verificando arquivos inválidos...


Verificando:   0%|          | 0/2 [00:00<?, ?it/s]Exception in thread Thread-5 (verificar_e_mover_arquivo):
Traceback (most recent call last):
  File "C:\Users\jaquelinerufino\.pyenv\pyenv-win\versions\3.12.0\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "c:\dev\Itau\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\jaquelinerufino\.pyenv\pyenv-win\versions\3.12.0\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jaquelinerufino\AppData\Local\Temp\ipykernel_19728\3268953147.py", line 3, in verificar_e_mover_arquivo
  File "c:\dev\Itau\.venv\Lib\site-packages\pandas\io\excel\_base.py", line 495, in read_excel
    io = ExcelFile(
         ^^^^^^^^^^
  File "c:\dev\Itau\.venv\Lib\site-packages\pandas\io\excel\_base.py", line 1550, in __init__
    ext = inspect_excel_format(
          ^^^^^^^^^^^^^^^^^^^^^
  File "c:\dev\Itau\.venv\Lib\site-pack

2. Renomeando arquivos...
3. Limpando planilhas...


Limpando: 0it [00:00, ?it/s]


4. Convertendo para PDF...


Convertendo: 0it [00:00, ?it/s]

✅ Processo finalizado.


In [ ]:
# Limpar cache do COM (apenas 1x por ambiente)
shutil.rmtree(os.path.join(win32com.__gen_path__), ignore_errors=True)

# Criar pasta de inválidos
os.makedirs(DIRETORIO_INVALIDO, exist_ok=True)

# 1. Verificar arquivos inválidos
arquivos = os.listdir(DIRETORIO)
threads = []
print("1. Verificando arquivos inválidos...")
for idx, arquivo in enumerate(tqdm(arquivos, desc="Verificando")):